### PreProcessing


In [ ]:

!unzip drive/'My Drive'/nnfl-assignment-2.zip

In [ ]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from keras.layers import  Flatten, Dense, Dropout


In [ ]:
!ls 

In [ ]:
test=pd.read_csv("./final_test.csv")
train=pd.read_csv("./final_train.csv")
print(train.head(5))

In [ ]:
x_train=train.drop(['rating'],axis=1)
x_class=[]
temp=np.array(train[['class']]).reshape(-1)
for i in temp:
#     print(i)
    a=np.zeros(836)
    a[i]=1
    x_class.append(a)
# print(min(np.array(train['class'])),max(np.array(train['class'])))
print(np.array(x_class).shape)
x_class=np.array(x_class)

y_train=train[['rating']]
train.info


In [ ]:
x_test=test
y_train=np.array(y_train).reshape((-1))
print(y_train)
y_train=np.array(y_train)

In [ ]:
print(y_train/10)
y_train=y_train/10

In [ ]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
num_words = 100000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ')
tokenizer.fit_on_texts(x_train['desc'].values)
X = tokenizer.texts_to_sequences(x_train['desc'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 200
X = pad_sequences(X, maxlen=max_length_of_text)

print(word_index)


In [ ]:
x_class.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X,y_train, test_size = 0.05, random_state = 42)
# print(X_train.shape,y_train.shape)
# X_train
# print(X_test.shape,y_test.shape)

### Creating Model

In [ ]:
import keras
embed_dim = 500 #Change to observe effects
lstm_out = 256 #Change to observe effects
batch_size = 32


input1 = keras.layers.Input(shape=(200,))

input2 = keras.layers.Input(shape=(836,))
# xb = keras.layers.Dense(8, activation='relu')(input2)
# print(inputs.shape)
x = Embedding(num_words, embed_dim)(input1)    # More info on Embeddings here - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
# # And here : https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer
x = LSTM(lstm_out)(x) # The LSTM transforms the vector sequence into a single vector of size lstm_out, containing information about the entire sequence
# print(classes.shape)
xb=Dense(512,activation='relu', kernel_initializer='normal')(input2)

x = Dense(512,activation='relu', kernel_initializer='normal')(x)
added = keras.layers.Concatenate(axis=-1)([x,xb])
c=Dropout(0.2)(added)
c = Dense(512,activation='relu', kernel_initializer='normal')(c)
c=Dropout(0.2)(c)
# print(added.shape)
c = Dense(256,activation='relu', kernel_initializer='normal')(c)
c=Dropout(0.2)(added)
c = Dense(128,activation='relu', kernel_initializer='normal')(c)
c=Dropout(0.2)(added)
c = Dense(64,activation='relu', kernel_initializer='normal')(c)

c = Dense(1,activation='relu', kernel_initializer='normal')(c)
# X=keras.layers.Input(X)
# x_class=keras.layers.Input(x_class)
model = Model(inputs=[input1, input2], outputs=c)
print(model.summary())
import keras

# input1 = keras.layers.Input(shape=(16,))
# x1 = keras.layers.Dense(8, activation='relu')(input1)
# input2 = keras.layers.Input(shape=(32,))
# x2 = keras.layers.Dense(8, activation='relu')(input2)
# # equivalent to added = keras.layers.add([x1, x2])
# added = keras.layers.Add()([x1, x2])

# out = keras.layers.Dense(4)(added)
# model = keras.models.Model(inputs=[input1, input2], outputs=out)


In [ ]:
import keras

# OPTIM=keras.optimizers.Adam(learning_rate=0.1, beta_1=0.999, beta_2=0.9999, amsgrad=False)
model.compile(loss = 'mean_squared_error', optimizer='adam',metrics = ['accuracy'])


In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="weights-improvement.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(x=[X,x_class], y=y_train, batch_size=1024, epochs=30,callbacks=callbacks_list, verbose=1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Y=tokenizer.texts_to_sequences(test['desc'].values)
testing_input = pad_sequences(Y, maxlen=max_length_of_text)
testing_classes=[]
temp=np.array(test[['class']])
for i in temp:
    a=np.zeros(836)
    a[i]=1
    testing_classes.append(a)
testing_classes=np.array(testing_classes)

In [ ]:
testing_classes.shape

In [ ]:
filepath

In [ ]:
model.load_weights("weights-improvement.hdf5")

In [ ]:
predict=model.predict([testing_input,testing_classes])

In [ ]:
print(predict[0:50])

### Post Processing of 3 types And creating submission file

In [ ]:
predictA=predict.reshape(-1)
predictB=predict.reshape(-1)

predictC=predict.reshape(-1)
predictD=predict.reshape(-1)


In [ ]:
predictA=predictA/np.max(predictA)
predictA*=10
for i in range(53175):
    if predictA[i]>10:
        predictA[i]=10.0
    elif predictA[i]<1:
        predictA[i]=1.0
    else:
        pass
y_test=np.ceil(predictA)
y_test.reshape(53175)
import csv
sub=open("sub1.csv","w")
writer=csv.writer(sub)
writer.writerow(["Id","rating"])
for i in range(0,53175):
    writer.writerow([i,y_test[i]])
sub.close()


<a href="sub1.csv">subA</a>

* <a href="weights-improvement.hdf5">download</a>

In [ ]:
# predictB=predictB
predictB*=10
for i in range(53175):
    if predictB[i]>10:
        predictB[i]=10.0
    elif predictB[i]<1:
        predictB[i]=1.0
    else:
        pass
y_test=np.ceil(predictB)
y_test.reshape(53175)
import csv
sub=open("sub2.csv","w")
writer=csv.writer(sub)
writer.writerow(["Id","rating"])
for i in range(0,53175):
    writer.writerow([i,y_test[i]])
sub.close()


<a href="sub2.csv">subB</a>

In [ ]:
predictC=predictC/np.max(predictC)
predictC*=10
predictC=np.floor(predictC)
for i in range(53175):
    if predictC[i]>10:
        predictC[i]=10.0
    elif predictC[i]<1:
        predictC[i]=1.0
    else:
        pass
# y_test=np.ceil(predictB)
y_test=y_test.reshape(-1)
import csv
sub=open("sub3.csv","w")
writer=csv.writer(sub)
writer.writerow(["Id","rating"])
for i in range(0,53175):
    writer.writerow([i,y_test[i]])
sub.close()


<a href="sub3.csv">subC</a>

In [ ]:
print('done')